## Importing Libraries

In [1]:
#Elsevier Libraries
from elsapy.elsclient import ElsClient
from elsapy.elsprofile import ElsAuthor, ElsAffil
from elsapy.elsdoc import FullDoc, AbsDoc
from elsapy.elssearch import ElsSearch

#Libraries for URL access
import json
import csv
import pprint
import requests
import xmltodict
import urllib3
import numpy as np

#Import warnings library
import warnings
warnings.filterwarnings("ignore")

In [2]:
#xml.etree.ElementTree — The ElementTree XML API
import xml.etree.ElementTree as ET

In [3]:
info_list = [] #Creating an empty list to store the  info
#Rerunning this will delete the data that will be stored further
#Run only once

In [4]:
#Create a num array of 0 to 5900 spaces 100 apart per page. Each page of Search returns 100 entries until 6000 is reached
num = np.linspace(0, 5900, 60, dtype = int)
print(num)
#Maximum of 6000 searches are returned on calling the ElsSearch API

[   0  100  200  300  400  500  600  700  800  900 1000 1100 1200 1300
 1400 1500 1600 1700 1800 1900 2000 2100 2200 2300 2400 2500 2600 2700
 2800 2900 3000 3100 3200 3300 3400 3500 3600 3700 3800 3900 4000 4100
 4200 4300 4400 4500 4600 4700 4800 4900 5000 5100 5200 5300 5400 5500
 5600 5700 5800 5900]


## Accessing the Links - API Extraction

In [5]:
#Append "apikey" and "insttoken" as suggest in the ElsSearch document into a config file
config = {
    "apikey": "2dc442325fc67f2f275ec3157ef8df65",
 "insttoken": "6beb1f6c29d85f50029bf11c8de94d1b"
    }

client = ElsClient(config['apikey'])
client.inst_token = config['insttoken']

In [6]:
def XML_DOI(link):
    
    #defining a header's dictionary to pass through requests
    headers_dict = {"X-ELS-APIKey": "2dc442325fc67f2f275ec3157ef8df65", "X-ELS-Insttoken": "6beb1f6c29d85f50029bf11c8de94d1b", "Accept": "application/xml"}
    
    #x takes response of the HTTP request, passes link
    x = requests.get(link, headers=headers_dict)
    
    #Save it as XML file
    with open("doi.xml", 'wb') as f:
        f.write(x.content)

In [9]:
link_list = []

#Loop to iterate over all papers
#UPI query can take many arguments.
#start, count, query are a few as name suggests
for i in range(0, 60):
    
    start = "https://api.elsevier.com/content/search/sciencedirect?" 
    count = "start=" + str(num[i]) + "&count=100"
    query = "&query=hydrogen+production+using+alcohol"
    endapi = "&apiKey=2dc442325fc67f2f275ec3157ef8df65&insttoken=6beb1f6c29d85f50029bf11c8de94d1b"
    
    link = start + count + query # + endapi 
    #print(link)
    link_list.append(link)
#subscription = "&subscribed=true"

In [11]:
link_list[6]

'https://api.elsevier.com/content/search/sciencedirect?start=600&count=100&query=hydrogen+production+using+alcohol'

## XML Parsing

In [12]:
# Call the XML_DOI function
for j in range(0, 60):
    xmlfile = XML_DOI(link_list[j])
    #Read the data
    #<?xml version="1.0" encoding="UTF-8"?>
    tree = ET.parse(r'C:\Users\Admin\OneDrive - IIT Delhi\CPCB\PROF. HARI\Jupyter_file\Ex_scibert\doi.xml')
    root = tree.getroot()
    for entry in root.findall('{http://www.w3.org/2005/Atom}entry'):
    
        info_dict = {} #This will create a dictionary where we will store information about the searches
    
        '''
        CHECK CODE
        url = entry.find('{http://prismstandard.org/namespaces/basic/2.0/}url').text
        title = entry.find('{http://purl.org/dc/elements/1.1/}title').text
        pub_name = entry.find('{http://prismstandard.org/namespaces/basic/2.0/}publicationName').text
        doi = entry.find('{http://prismstandard.org/namespaces/basic/2.0/}doi').text
        #description = entry.find('{http://purl.org/dc/elements/1.1/}description').text
        print(url, title, pub_name, doi)
        print('\n')
        
        '''
    
        info_dict['URL'] = entry.find('{http://prismstandard.org/namespaces/basic/2.0/}url').text
        info_dict['Title'] = entry.find('{http://purl.org/dc/elements/1.1/}title').text
        info_dict['Pub_Name'] = entry.find('{http://prismstandard.org/namespaces/basic/2.0/}publicationName').text
        doi = entry.find('{http://prismstandard.org/namespaces/basic/2.0/}doi')
        if doi is None:
            info_dict['DOI'] = None
        else:
            info_dict['DOI'] = doi.text
    
        info_list.append(info_dict)
    
    #print(info_list)

In [20]:
type(info_list), len(info_list)

(list, 6000)

In [25]:
DOI =  []
Title = []
Pub_name = []

for i in range(6000):
    DOI.append(info_list[i]['DOI'])
    Title.append(info_list[i]['Title'])
    Pub_name.append(info_list[i]['Pub_Name'])
      
    

In [26]:
df =  pd.DataFrame()
df['Title'] =  Title
df['Pub_name'] = Pub_name
df['DOI'] = DOI

df.head()

,Title,Pub_name,DOI
0,Hydrogen and alcohols production by <ce:italic...,Journal of CO2 Utilization,10.1016/j.jcou.2022.101914
1,Photocatalytic hydrogen production from alcoho...,Journal of Photochemistry and Photobiology A: ...,10.1016/j.jphotochem.2021.113726
2,Homogeneous first-row transition metal catalys...,Tetrahedron,10.1016/j.tet.2021.132473
3,Selective photoelectrocatalytic tuning of benz...,Applied Catalysis B: Environmental,10.1016/j.apcatb.2020.119868
4,Light alcohols reforming towards renewable hyd...,Renewable and Sustainable Energy Reviews,10.1016/j.rser.2020.110523


In [27]:
#Create string1 and string2 to join doi with institoken to make a single URL
string1 = "https://api.elsevier.com/content/article/doi/" 
string2 = "?apiKey=2dc442325fc67f2f275ec3157ef8df65&insttoken=6beb1f6c29d85f50029bf11c8de94d1b"

#Access every DOI in the previous file and append the new URL to another column
df['Link'] = df['DOI'].apply(lambda x: string1 + str(x) + string2)

df.head()

,Title,Pub_name,DOI,Link
0,Hydrogen and alcohols production by <ce:italic...,Journal of CO2 Utilization,10.1016/j.jcou.2022.101914,https://api.elsevier.com/content/article/doi/1...
1,Photocatalytic hydrogen production from alcoho...,Journal of Photochemistry and Photobiology A: ...,10.1016/j.jphotochem.2021.113726,https://api.elsevier.com/content/article/doi/1...
2,Homogeneous first-row transition metal catalys...,Tetrahedron,10.1016/j.tet.2021.132473,https://api.elsevier.com/content/article/doi/1...
3,Selective photoelectrocatalytic tuning of benz...,Applied Catalysis B: Environmental,10.1016/j.apcatb.2020.119868,https://api.elsevier.com/content/article/doi/1...
4,Light alcohols reforming towards renewable hyd...,Renewable and Sustainable Energy Reviews,10.1016/j.rser.2020.110523,https://api.elsevier.com/content/article/doi/1...


In [29]:
#Save the combined abstracts in another CSV file
df.to_csv(r'C:\Users\Admin\OneDrive - IIT Delhi\CPCB\PROF. HARI\Jupyter_file\Ex_scibert\doi_6000_withlink.csv', index=False)